# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

### 10 Countries with the lowest infant mortality rates

In [6]:
import numpy as np
import pandas as pd

In [7]:
# First, get to the root
root = document.getroot()

# Loop over all the countries
country_by_infant_mortality = {}
for cntr in root.iterfind('country'):
    # Get the infant mortality in each country
    name = cntr.find('name').text
    try:
        country_by_infant_mortality[name] = float(cntr.find('infant_mortality').text)
    except:
        country_by_infant_mortality[name] = np.nan
    
# Convert to series
ser1 = pd.Series(country_by_infant_mortality)

# Compute lowest 10
ser1.sort_values().head(10)

Monaco            1.81
Japan             2.13
Norway            2.48
Bermuda           2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
dtype: float64

### 10 Cities with the largest populations


In [8]:
# Loop over all the countries

cities_by_population = {}
for cntr in root.iterfind('country'):
    # Get the cities
    for city in cntr.findall('city'):
        # Get the latest population
        name = city.find('name').text
        try:
            cities_by_population[name] = float(city.findall('population')[-1].text)
        except:
            cities_by_population[name] = np.nan
            
# Convert to series
ser2 = pd.Series(cities_by_population)

# Compute top 10
ser2.sort_values(ascending=False).head(10)

Seoul              9708483.0
Al Qahirah         8471859.0
Bangkok            7506700.0
Hong Kong          7055071.0
Ho Chi Minh        5968384.0
Singapore          5076700.0
Al Iskandariyah    4123869.0
New Taipei         3939305.0
Busan              3403135.0
Pyongyang          3255288.0
dtype: float64

### 10 Ethnic groups with largest populations

In [9]:
from collections import Counter
ethnic_groups_by_population = Counter()

for cntr in root.iterfind('country'):
    # Get the population of the country (the latest record)
    pop = float(cntr.findall('population')[-1].text)
    
    # Now loop over all the ethnic groups
    list_of_eth = cntr.findall('ethnicgroup')
    if (len(list_of_eth) > 0):
        for eth in list_of_eth:
            # Get precentage
            perc = eth.attrib['percentage']
            ethnic_groups_by_population[eth.text] += pop * float(perc)

# Use Counter's most common method
ethnic_groups_by_population.most_common(10)

[('Han Chinese', 124505880000.0),
 ('Indo-Aryan', 87181558344.0),
 ('European', 49487221971.96),
 ('African', 31832512036.9),
 ('Dravidian', 30271374425.0),
 ('Mestizo', 15773435493.7),
 ('Bengali', 14677691672.0),
 ('Russian', 13185699607.699999),
 ('Japanese', 12653421200.0),
 ('Malay', 12199355037.4)]

### Name & Country of a) longest river, b) largest lake, c) airport at highest elevation

In [11]:
from collections import defaultdict

# First let's get a list of all countries, rivers, lakes and airports
countries = root.findall('country')
rivers = root.findall('river')
lakes = root.findall('lake')
airports = root.findall('airport')

# Names of countries will be macthed with their abbreviations (or car codes)
cntr_abbrv = {}
for cntr in countries:
    full_name = cntr.find('name').text
    abbrv = cntr.attrib['car_code']
    cntr_abbrv[abbrv] = full_name

# Convert into dataframe to be merged later
df_abbrv = pd.Series(cntr_abbrv).to_frame().reset_index()
df_abbrv.index = range(len(cntr_abbrv))
df_abbrv.columns = ['abbrv_country', 'country_name']

Longest River by Country

In [12]:
# Longest river
longest_rivers_by_country = defaultdict(list) # Initiate
for river in rivers:
    cntrs = river.attrib['country'].split() # Name(s) of the country where the river is located
    river_name = river.find('name').text # Name of river
    # Length of river
    try:
        length = float(river.find('length').text)
    except:
        length = np.nan
    
    # Add to the dictionary
    for cntr in cntrs:
        # Store in the dictionary
        longest_rivers_by_country['river_name'].append(river_name)
        longest_rivers_by_country['abbrv_country'].append(cntr)
        longest_rivers_by_country['river_length'].append(length)

# Now convert into pandas dataframe
df1 = pd.DataFrame(longest_rivers_by_country)

# Merge with abbreviations
df1 = df1.merge(df_abbrv, on = "abbrv_country")

# Now sort by river length and print
df1.sort_values('river_length', ascending=False)[['river_name', 'river_length', 'country_name']].head(5)


,river_name,river_length,country_name
310,Amazonas,6448.0,Peru
299,Amazonas,6448.0,Colombia
302,Amazonas,6448.0,Brazil
226,Jangtse,6380.0,China
225,Hwangho,4845.0,China


In [13]:
# We can also extract the longest river countries by groupby
grps = df1.groupby(['river_name', 'country_name'])
df2 = grps.apply(lambda x: np.max(x.river_length)).sort_values(ascending = False)
df2[df2.index[0][0]]

country_name
Brazil      6448.0
Peru        6448.0
Colombia    6448.0
dtype: float64

So, the longest river is the Amazon river, which run through Brazil, Peru and Colombia.

Largest lake by Country

In [15]:
# Largest lake
largest_lakes_by_country = defaultdict(list) # Initiate
for lake in lakes:
    cntrs = lake.attrib['country'].split() # Name(s) of the country where the lake is located
    lake_name = lake.find('name').text # Name of lake
    # Length of river
    try:
        length = float(lake.find('area').text)
    except:
        length = np.nan
    
    # Add to the dictionary
    for cntr in cntrs:
        # Store in the dictionary
        largest_lakes_by_country['lake_name'].append(lake_name)
        largest_lakes_by_country['abbrv_country'].append(cntr)
        largest_lakes_by_country['lake_area'].append(length)

# Now convert into pandas dataframe
df3 = pd.DataFrame(largest_lakes_by_country)

# Merge with abbreviations
df3 = df3.merge(df_abbrv, on = "abbrv_country")

# Now sort by river length and print
df3.sort_values('lake_area', ascending=False)[['lake_name', 'lake_area', 'country_name']].head(6)


,lake_name,lake_area,country_name
56,Caspian Sea,386400.0,Russia
72,Caspian Sea,386400.0,Iran
73,Caspian Sea,386400.0,Azerbaijan
74,Caspian Sea,386400.0,Kazakhstan
77,Caspian Sea,386400.0,Turkmenistan
148,Lake Superior,82103.0,Canada


In [16]:
# Now by groupby
grps = df3.groupby(['lake_name', 'country_name'])
df4 = grps.apply(lambda x: np.max(x.lake_area)).sort_values(ascending = False)
df4[df4.index[0][0]]

country_name
Kazakhstan      386400.0
Iran            386400.0
Azerbaijan      386400.0
Russia          386400.0
Turkmenistan    386400.0
dtype: float64

So, the largest lake is the Caspian Sea, which is located in the above countries.

Airport with highest elevation

In [17]:
# Highest elevation airport
airports_by_country = defaultdict(list) # Initiate
for airport in airports:
    cntr = airport.attrib['country'] # Name of the country where the lake is located
    airport_name = airport.find('name').text
    try:
        elevation = float(airport.find('elevation').text)
    except:
        elevation = np.nan
    
    # Add to dictionary
    airports_by_country['airport_name'].append(airport_name)
    airports_by_country['abbrv_country'].append(cntr)
    airports_by_country['elevation'].append(elevation)
    
# Now convert into pandas dataframe
df5 = pd.DataFrame(airports_by_country)

# Merge with abbreviations
df5 = df5.merge(df_abbrv, on = "abbrv_country")

# Now sort by river length and print
df5.sort_values('elevation', ascending=False)[['airport_name', 'elevation', 'country_name']].head(5)

,airport_name,elevation,country_name
80,El Alto Intl,4063.0,Bolivia
219,Lhasa-Gonggar,4005.0,China
241,Yushu Batang,3963.0,China
813,Juliaca,3827.0,Peru
815,Teniente Alejandro Velasco Astete Intl,3311.0,Peru


So, the highest elevation airport is El Alto Intl located at Bolivia.